In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

df = pd.read_csv("netflix_reviews.csv")  # 파일 불러오기

# 전처리 함수
def preprocess_text(text):
    if isinstance(text, float):
        return ""
    text = text.lower()  # 대문자를 소문자로
    text = re.sub(r'[^\w\s]', '', text)  # 구두점 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = text.strip()  # 띄어쓰기 제외하고 빈 칸 제거
    return text

# content 컬럼에 전처리 함수 적용
df['content'] = df['content'].apply(preprocess_text)

# 필요한 열 선택 및 결측값 처리
df = df[['content', 'score']].dropna().reset_index(drop=True)
df = df[df['score'].isin([1,2,3,4,5])]

In [3]:
# 특성과 타겟 분리
X = df['content']
y = df['score']

import torch.nn.utils.rnn as rnn_utils

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, reviews, ratings, text_pipeline):
        self.reviews = reviews.reset_index(drop=True)
        self.ratings = ratings.reset_index(drop=True)
        self.text_pipeline = text_pipeline
        # self.label_pipeline = label_pipeline

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.text_pipeline(self.reviews.iloc[idx])
        rating = self.ratings.iloc[idx]
        rating_tensor = torch.tensor(rating, dtype=torch.float)
        return review, rating_tensor

# 데이터 분할
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = get_tokenizer('basic_english')
# basic english를 기반으로 한 토큰 함수를 tokenizer로 선언
# tokens = tokenizer("This is a sample sentence.")
# >> 이제 이 코드의 결과는 ['this', 'is', 'a', 'sample', 'sentence']와 같은 형태의 리스트를 tokens에 저장한다

# 어휘 생성 함수
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)
# 문장으로 이루어진 list(data_iter)가 입력되면, 각 문장마다 단어 형태로 분리해서 리스트로 묶은걸 반환해준다.

# train_reviews에 있는 모든 문장들이 토큰화되어 사전 형태로 vocab에 추가된다.
# 첫번째 값으로 <unk> 을 저장한다. 이 때, 각 단어마다 인덱스가 부여된다.
# 예시) vocab = {"<unk>": 0, "this": 1, "is": 2, "a": 3, "great": 4, "test": 5}
vocab = build_vocab_from_iterator(yield_tokens(train_reviews), specials=["<unk>"])

#vocab에 모르는 단어를 입력받을 경우, <unk>으로 저장한다.
vocab.set_default_index(vocab["<unk>"])

# vocab.get_itos() 를 통해 확인 가능
print(vocab['netflix'])  # 8
print(vocab['is'])       # 6
print(vocab['great'])    # 44
print(vocab['ott'])      # 711
print(vocab['platform']) # 320

8
6
44
711
320


In [4]:
# 텍스트 파이프라인 정의
def text_pipeline(text):
    return torch.tensor([vocab[token] for token in tokenizer(text)], dtype=torch.long)

# 레이블 파이프라인 정의
# cat -> 0, dog -> 1, bird -> 2와 같은 식으로 카테고리를 숫자로 변환하는 것을 레이블 인코딩
# label_encoder = LabelEncoder() # 라벨인코더 함수 생성
# label_encoder.fit(df['score'].unique())  # 예시 레이블
# def label_pipeline(label):
#     return label_encoder.transform([label])[0]

# 데이터 로더 정의
BATCH_SIZE = 64

# collate 함수 정의
def collate_review_rating(batch):
    reviews, ratings = zip(*batch)  # 배치에서 리뷰와 레이블 분리
    reviews_padded = torch.nn.utils.rnn.pad_sequence(reviews, batch_first=True)  # 패딩 적용
    ratings_tensor = torch.stack(ratings)  # 레이블은 스택
    return reviews_padded, ratings_tensor

# 데이터셋 정의
train_dataset = ReviewDataset(train_reviews, train_ratings, text_pipeline)
test_dataset = ReviewDataset(test_reviews, test_ratings, text_pipeline)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_review_rating)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_review_rating)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()                             # 상속 클래스(Module에 대한 초기화 실행)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden[-1])

# 하이퍼파라미터 정의
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = 1  # 예측할 점수 개수

# 모델 초기화
model = LSTMModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)

# 손실 함수와 옵티마이저 정의
# criterion = nn.CrossEntropyLoss() # 분류 문제에 적합
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [8]:
# GPU 가동이 가능한 경우(CUDA), GPU 로 구동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
model = model.to(device)
criterion = criterion.to(device)

# 모델 학습/평가
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0
    test_loss = 0
    
    # 학습 데이터 로드
    model.train()
    for reviews_padded, ratings_tensor in train_dataloader:
        #GPU 변환
        reviews_padded = reviews_padded.to(device)
        ratings_tensor = ratings_tensor.to(device)

        optimizer.zero_grad()
        outputs = model(reviews_padded)
        ratings_tensor = ratings_tensor.unsqueeze(1)
        loss = criterion(outputs, ratings_tensor)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 평가 데이터 로드
    model.eval()
    with torch.no_grad():
        for reviews_padded, ratings_tensor in test_dataloader:
            #GPU 변환
            reviews_padded = reviews_padded.to(device)
            ratings_tensor = ratings_tensor.to(device)

            outputs = model(reviews_padded)
            ratings_tensor = ratings_tensor.unsqueeze(1)
            loss = criterion(outputs, ratings_tensor)
            test_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_dataloader):.4f}, Test Loss: {test_loss/len(test_dataloader):.4f}')

print('Finished Training')

Epoch [1/10], Train Loss: 0.9917, Test Loss: 1.0793
Epoch [2/10], Train Loss: 0.9648, Test Loss: 1.1027
Epoch [3/10], Train Loss: 0.9247, Test Loss: 1.0726
Epoch [4/10], Train Loss: 0.9149, Test Loss: 1.0686
Epoch [5/10], Train Loss: 0.8977, Test Loss: 1.0550
Epoch [6/10], Train Loss: 0.8913, Test Loss: 1.0542
Epoch [7/10], Train Loss: 0.8805, Test Loss: 1.0753
Epoch [8/10], Train Loss: 0.8867, Test Loss: 1.0684
Epoch [9/10], Train Loss: 0.8766, Test Loss: 1.0797
Epoch [10/10], Train Loss: 0.8821, Test Loss: 1.0664
Finished Training


In [9]:
# 예측 함수(예시)
def predict_review(model, new_review):
    model.eval()
    with torch.no_grad():
        review_tensor = text_pipeline(new_review)
        # review_tensor = review_tensor.unsqueeze(0)  # 배치 차원 추가
        output = model(review_tensor)
        # prediction = output.argmax(1).item()
        return output.item()
    
# 새로운 리뷰에 대한 예측 (1)
new_review = "It's very amazing ott platform. Almost every single contents are so creative and awesome."
print(f'Predicted Score: {predict_review(model, new_review)}')

# 새로운 리뷰에 대한 예측 (2)
new_review = "It's so terrible. I couldn't apply 4K UHD although I paid premium membership."
print(f'Predicted Score: {predict_review(model, new_review)}')

# 새로운 리뷰에 대한 예측 (3)
new_review = "I couldn't find some movies, but Netflix was good overall."
print(f'Predicted Score: {predict_review(model, new_review)}')

Predicted Score: 4.8705549240112305
Predicted Score: 1.2798094749450684
Predicted Score: 3.95182728767395
